In this practice code, we are going to use Ozone dataset. Dictionary of this dataset can be found in https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/airquality.html

In [31]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [48]:
url = "https://raw.githubusercontent.com/ga-students/DS-SF-24/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)
OzoneData.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day
count,116.000000,146.000000,153.000000,153.000000,153.000000,153.000000
mean,42.129310,185.931507,9.957516,77.882353,6.993464,15.803922
std,32.987885,90.058422,3.523001,9.465270,1.416522,8.864520
min,1.000000,7.000000,1.700000,56.000000,5.000000,1.000000
25%,18.000000,115.750000,7.400000,72.000000,6.000000,8.000000
50%,31.500000,205.000000,9.700000,79.000000,7.000000,16.000000
75%,63.250000,258.750000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


#### Explore the dataset and decide which variables suffer from missing data

In [33]:
OzoneData.head(20)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5
5,28.0,NaN,14.9,66,5,6
6,23.0,299.0,8.6,65,5,7
7,19.0,99.0,13.8,59,5,8
8,8.0,19.0,20.1,61,5,9
9,NaN,194.0,8.6,69,5,10


Answer: Ozone and Solar.R

#### Let's drop rows that have missing values in all the columns you indentified above

Hint: in dropna() if you set how = 'all', it will only drop columns that are suffering from missing values at all varaibles you introduce in subset. If you want to get rid of the row that contains missing values in any of the variables you specify, then you shall set how = 'any'

df.dropna(how = 'all',subset = ['Var1','Var2','Var3'],inplace = True)

The above code will check if all 3 variables specified in df have missing values, if they all have missing values it will drop that row.

In [34]:
len(OzoneData)

153

In [35]:
OzoneData.dropna(how = 'all', subset = ['Ozone','Solar.R'], inplace = True)

In [36]:
len(OzoneData)

151

#### Spoiler! If everything is going according to plan you shall be left by 151 observations. Also, it seemed like the first two varibales had missing values. Now please make a copy of your dataframe into a dataframe named OzoneImputeMean. Also, please use mean of the variables to fill in missing values in OzoneImputeMean.

Hint: copy OzoneData this way:

OzoneImputeMean = OzoneData.copy()


In [37]:
OzoneImputeMean = OzoneData.copy()

OzoneImputeMean['Ozone'].fillna(value = np.mean(OzoneImputeMean['Ozone']), inplace = True)
OzoneImputeMean['Solar.R'].fillna(value = np.mean(OzoneImputeMean['Solar.R']), inplace = True)

OzoneImputeMean.head(20)

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.00000,190.000000,7.4,67,5,1
1,36.00000,118.000000,8.0,72,5,2
2,12.00000,149.000000,12.6,74,5,3
3,18.00000,313.000000,11.5,62,5,4
5,28.00000,185.931507,14.9,66,5,6
6,23.00000,299.000000,8.6,65,5,7
7,19.00000,99.000000,13.8,59,5,8
8,8.00000,19.000000,20.1,61,5,9
9,42.12931,194.000000,8.6,69,5,10
10,7.00000,185.931507,6.9,74,5,11


In [38]:
#Check if missing values are filled
len(OzoneImputeMean[OzoneImputeMean['Ozone'].isnull() == True]) # 0 
len(OzoneImputeMean[OzoneImputeMean['Ozone'].isnull() == False]) # 151

151

#### Now it's time for imputing using linear regression lines

In [39]:
#### Before we start let's define dummy variables for variable Month - don't worry about day!

MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']

OzoneData = pd.concat([OzoneData, MonthDummy], axis=1)

OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41.0,190.0,7.4,67,5,1,1.0,0.0,0.0,0.0
1,36.0,118.0,8.0,72,5,2,1.0,0.0,0.0,0.0
2,12.0,149.0,12.6,74,5,3,1.0,0.0,0.0,0.0
3,18.0,313.0,11.5,62,5,4,1.0,0.0,0.0,0.0
5,28.0,NaN,14.9,66,5,6,1.0,0.0,0.0,0.0


In [40]:
#now let's explore correlation Matrix
OzoneData.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.601547,0.698360,0.164519,-0.013226,-0.302963,-0.112006,0.277961,0.291808
Solar.R,0.348342,1.000000,-0.056792,0.275840,-0.075301,-0.150275,-0.024601,0.023998,0.176744,-0.076390
Wind,-0.601547,-0.056792,1.000000,-0.466032,-0.175317,0.042365,0.237781,0.046054,-0.144638,-0.166105
Temp,0.698360,0.275840,-0.466032,1.000000,0.397427,-0.136876,-0.637816,0.050753,0.318103,0.321680
Month,0.164519,-0.075301,-0.175317,0.397427,1.000000,-0.007727,-0.702257,-0.362131,-0.007201,0.355246
Day,-0.013226,-0.150275,0.042365,-0.136876,-0.007727,1.000000,0.011003,-0.017044,0.011471,0.011471
Month_5,-0.302963,-0.024601,0.237781,-0.637816,-0.702257,0.011003,1.000000,-0.242766,-0.247805,-0.247805
Month_6,-0.112006,0.023998,0.046054,0.050753,-0.362131,-0.017044,-0.242766,1.000000,-0.253080,-0.253080
Month_7,0.277961,0.176744,-0.144638,0.318103,-0.007201,0.011471,-0.247805,-0.253080,1.000000,-0.258333
Month_8,0.291808,-0.076390,-0.166105,0.321680,0.355246,0.011471,-0.247805,-0.253080,-0.258333,1.000000


#### Create 3 datasets. Let's Name the first one OzoneData_NoMissing which contains no missing values. DataSet 2 is called OzoneData_SolarMissingDrop which does not have any Solar.R missing value. DataSet 3 is named OzoneData_OzoneMissingDrop which does not have any Ozone Missing Value. 

Hint: 
df2 = df1.copy() only copies df1 without changing anything in df1
for instance, here we create OzoneData_NoMissing

OzoneData_NoMissing = OzoneData.dropna(how = 'any', subset = ['Ozone', 'Solar.R'], inplace = False).copy()


In [41]:

OzoneData_NoMissing = OzoneData.dropna(how = 'any', subset = ['Ozone', 'Solar.R'], inplace = False).copy()
OzoneData_SolarMissingDrop = OzoneData.dropna(how = 'any', subset = ['Solar.R'], inplace = False).copy()
OzoneData_OzoneMissingDrop = OzoneData.dropna(how = 'any', subset = ['Ozone'], inplace = False).copy()

#### What seems to be the list of best variables can define Ozone? how about Solar.R?

Answer: 

Ozone => Wind, Temp, Solar.R 
Solar.R => Ozone, Temp

#### Now let's use a regression model to predict Ozone. First drop NaN values in Ozone and save it in OzoneDroppedValues_Ozone. Then run a regression line on variables of interest and check significancy of your model if in a multi class dummy variable case, you see only a few of the dummy variables are not significant but the majority are,  you shall either drop all or keep all. Otherwise, selecting the base dummy will become important use these variables ['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8'] to predict. 

Hint: For this part you need to use 

X = OzoneData_NoMissing[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]

y = OzoneData_NoMissing['Ozone']

In [42]:
X = OzoneData_NoMissing[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
y = OzoneData_NoMissing['Ozone']

regression_line = smf.ols(formula = 'y ~ X', data = OzoneData_NoMissing).fit()
print(regression_line.pvalues)

Intercept    2.318427e-03
X[0]         2.957179e-02
X[1]         7.777659e-06
X[2]         2.740750e-07
X[3]         1.823435e-02
X[4]         8.812922e-01
X[5]         2.289481e-01
X[6]         5.824286e-02
dtype: float64


In [43]:
# now fill in null values of OzoneData['Ozone'] by predicted values
# Here you can see the guideline to fill in missing values of Ozone. Next 
linreg = LinearRegression()

#### Run your model
X = OzoneData_NoMissing[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
y = OzoneData_NoMissing['Ozone']
linreg.fit(X, y)

#Pay attention how we make predictions
X2 = OzoneData_SolarMissingDrop[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
OzoneData_SolarMissingDrop['Ozone_predicted'] = linreg.predict(X2)
OzoneData['Ozone'].fillna(value = OzoneData_SolarMissingDrop['Ozone_predicted'], inplace = True)

In [44]:
# Now repeat previous steps for Solar.R variable using ['Ozone','Wind','Temp]


#Check p-values of your model - if a variable is not significant, drop it
X = OzoneData_NoMissing[['Ozone','Wind','Temp']]
y = OzoneData_NoMissing['Solar.R']

regression_line = smf.ols(formula = 'y ~ X', data = OzoneData_NoMissing).fit()
print(regression_line.pvalues)

Intercept    0.867969
X[0]         0.011237
X[1]         0.188555
X[2]         0.345449
dtype: float64


In [45]:
# now fill in null values of OzoneData['Solar.R'] by predicted values
linreg = LinearRegression()

X = OzoneData_NoMissing[['Ozone']]
y = OzoneData_NoMissing['Solar.R']

linreg.fit(X, y)

X2 = OzoneData_OzoneMissingDrop[['Ozone']]
OzoneData_OzoneMissingDrop['Solar_predicted'] = linreg.predict(X2)
OzoneData['Solar.R'].fillna(value = OzoneData_OzoneMissingDrop['Solar_predicted'], inplace = True)

In [46]:
OzoneData.head(20)

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41.000000,190.000000,7.4,67,5,1,1.0,0.0,0.0,0.0
1,36.000000,118.000000,8.0,72,5,2,1.0,0.0,0.0,0.0
2,12.000000,149.000000,12.6,74,5,3,1.0,0.0,0.0,0.0
3,18.000000,313.000000,11.5,62,5,4,1.0,0.0,0.0,0.0
5,28.000000,171.348349,14.9,66,5,6,1.0,0.0,0.0,0.0
6,23.000000,299.000000,8.6,65,5,7,1.0,0.0,0.0,0.0
7,19.000000,99.000000,13.8,59,5,8,1.0,0.0,0.0,0.0
8,8.000000,19.000000,20.1,61,5,9,1.0,0.0,0.0,0.0
9,38.543618,194.000000,8.6,69,5,10,1.0,0.0,0.0,0.0
10,7.000000,151.310012,6.9,74,5,11,1.0,0.0,0.0,0.0


#### Now check your filled data - if your predicted values are more than maximum or less than minimum, replace them by max and min

In [47]:
OzoneData.describe() # Ozone min used to be 1.00 and now it's -15.511696

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,41.908287,185.916245,9.941722,78.165563,7.019868,15.801325,0.192053,0.198675,0.205298,0.205298
std,30.537618,88.659136,3.524984,9.198138,1.406984,8.832531,0.395225,0.400331,0.405263,0.405263
min,-15.511696,7.000000,1.700000,57.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,19.500000,119.000000,7.400000,73.000000,6.000000,8.000000,0.000000,0.000000,0.000000,0.000000
50%,36.000000,203.000000,9.700000,79.000000,7.000000,16.000000,0.000000,0.000000,0.000000,0.000000
75%,57.763947,257.000000,11.500000,85.000000,8.000000,23.000000,0.000000,0.000000,0.000000,0.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000,1.000000,1.000000,1.000000,1.000000


#### Now chack your dataset and see if imputed values are in acceptable range. If there is anything wrong with it just point it out. Fix "Out of range" values if find any.

Answer: 

In [50]:
OzoneData.loc[OzoneData.Ozone < 1, 'Ozone'] = 1
OzoneData.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day
count,116.000000,146.000000,153.000000,153.000000,153.000000,153.000000
mean,42.129310,185.931507,9.957516,77.882353,6.993464,15.803922
std,32.987885,90.058422,3.523001,9.465270,1.416522,8.864520
min,1.000000,7.000000,1.700000,56.000000,5.000000,1.000000
25%,18.000000,115.750000,7.400000,72.000000,6.000000,8.000000
50%,31.500000,205.000000,9.700000,79.000000,7.000000,16.000000
75%,63.250000,258.750000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


#### Bonus: Repeat the above procedure, this time fill in missing values using regression with errors. 

In [67]:
url = "https://raw.githubusercontent.com/ga-students/DS-SF-24/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)

MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']

OzoneData = pd.concat([OzoneData, MonthDummy], axis=1)
OzoneData_NoMissing = OzoneData.dropna(how = 'any', subset = ['Ozone', 'Solar.R'], inplace = False).copy()

In [69]:
linreg = LinearRegression()

X = OzoneData_NoMissing[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]
y = OzoneData_NoMissing['Ozone']
linreg.fit(X, y)

y_hat = linreg.predict(X)

Standard_error = ( sum( (y - y_hat) ** 2 )/(len(y) - 2) ) ** .5

y_min = y.min()
y_max = y.max()
OzoneData_SolarMissingDrop = OzoneData.dropna(how = 'any', subset = ['Solar.R'], inplace = False).copy()
X = OzoneData_SolarMissingDrop[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]

OzoneData_SolarMissingDrop['predicted_ozone_error'] = linreg.predict(X) + np.random.normal(0, Standard_error, len(OzoneData))

print('Minimum Values before adjustment %f:'  %OzoneData['predicted_ozone_error'].min())

OzoneData.loc[(OzoneData['predicted_ozone_error'] < y_min), 'predicted_ozone_error'] = y_min 
OzoneData.loc[(OzoneData['predicted_ozone_error'] > y_max), 'predicted_ozone_error'] = y_max

print('Minimum Values after adjustment %f:'  %OzoneData['predicted_ozone_error'].min())


OzoneData['Ozone'].fillna(value = OzoneData['predicted_ozone_error'], inplace = True)


OzoneData.head(20)

ValueError: operands could not be broadcast together with shapes (146,) (153,) 